<a href="https://colab.research.google.com/github/DineshIT/DineshIT/blob/main/notebooks/gradio_demos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Re-Aging Demos with Gradio
To try out the Gradio Demos using Google Colab, run the cells below.
Be sure to select a runtime with GPU (e.g. `Runtime > Change Runtime Type > T4 (GPU)`) to make the model run fast.

You can choose to either run the regular inference demo or the video demo. The demo will open in a new tab.

## Downloading files and model

In [1]:
!git clone https://github.com/timroelofs123/face_reaging

Cloning into 'face_reaging'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 139 (delta 23), reused 21 (delta 10), pack-reused 88 (from 1)
Receiving objects: 100% (139/139), 48.26 MiB | 19.67 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [2]:
!git clone https://huggingface.co/timroelofs123/face_re-aging

Cloning into 'face_re-aging'...
remote: Enumerating objects: 11, done.
remote: Total 11 (delta 0), reused 0 (delta 0), pack-reused 11 (from 1)
Unpacking objects: 100% (11/11), 3.27 KiB | 1.09 MiB/s, done.


In [3]:
%mv face_re-aging/best_unet_model.pth face_reaging/
%cd face_reaging/

/content/face_reaging


## Installing requirements

In [4]:
%pip install gradio face_recognition antialiased_cnns kaleido av --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.4 MB/s eta 0:00:00


## Running demo

Re-age your photo or video with the three demos.

In [5]:
import gradio as gr
import torch
from model.models import UNet
from scripts.test_functions import process_image, process_video

In [6]:
# default settings
window_size = 512
stride = 256
steps = 18
frame_count = 100
model_path = 'best_unet_model.pth'

In [7]:
# The code from this block equivalent to the "run" function in scripts.gradio_demo

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
unet_model = UNet().to(device)
unet_model.load_state_dict(torch.load(model_path, map_location=device))
unet_model.eval()

def block_img(image, source_age, target_age):
    return process_image(unet_model, image, video=False, source_age=source_age,
                          target_age=target_age, window_size=window_size, stride=stride)

def block_img_vid(image, source_age):
    return process_image(unet_model, image, video=True, source_age=source_age,
                          target_age=0, window_size=window_size, stride=stride, steps=steps)

def block_vid(video_path, source_age, target_age):
    return process_video(unet_model, video_path, source_age, target_age,
                          window_size=window_size, stride=stride, frame_count=frame_count)

demo_img = gr.Interface(
    fn=block_img,
    inputs=[
        gr.Image(type="pil"),
        gr.Slider(10, 90, value=20, step=1, label="Current age", info="Choose your current age"),
        gr.Slider(10, 90, value=80, step=1, label="Target age", info="Choose the age you want to become")
    ],
    outputs="image",
    examples=[
        ['assets/gradio_example_images/1.png', 20, 80],
        ['assets/gradio_example_images/2.png', 75, 40],
        ['assets/gradio_example_images/3.png', 30, 70],
        ['assets/gradio_example_images/4.png', 22, 60],
        ['assets/gradio_example_images/5.png', 28, 75],
        ['assets/gradio_example_images/6.png', 35, 15]
    ],
    description="Input an image of a person and age them from the source age to the target age."
)

demo_img_vid = gr.Interface(
    fn=block_img_vid,
    inputs=[
        gr.Image(type="pil"),
        gr.Slider(10, 90, value=20, step=1, label="Current age", info="Choose your current age"),
    ],
    outputs=gr.Video(),
    examples=[
        ['assets/gradio_example_images/1.png', 20],
        ['assets/gradio_example_images/2.png', 75],
        ['assets/gradio_example_images/3.png', 30],
        ['assets/gradio_example_images/4.png', 22],
        ['assets/gradio_example_images/5.png', 28],
        ['assets/gradio_example_images/6.png', 35]
    ],
    description="Input an image of a person and a video will be returned of the person at different ages."
)

demo_vid = gr.Interface(
    fn=block_vid,
    inputs=[
        gr.Video(),
        gr.Slider(10, 90, value=20, step=1, label="Current age", info="Choose your current age"),
        gr.Slider(10, 90, value=80, step=1, label="Target age", info="Choose the age you want to become")
    ],
    outputs=gr.Video(),
    examples=[
        ['assets/gradio_example_images/orig.mp4', 35, 60],
    ],
    description="Input a video of a person, and it will be aged frame-by-frame."
)

demo = gr.TabbedInterface([demo_img, demo_img_vid, demo_vid],
                          tab_names=['Image inference demo', 'Image animation demo', 'Video inference demo'],
                          title="Face Re-Aging Demo",
                          )

<ipython-input-7-e8b46674c5af>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unet_model.load_state_dict(torch.load(model_path, map_location=device))


In [8]:
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://511b9d61a6deee457d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
